# Exercise 2: Advanced Analytics NLP

In [ ]:
!pip install spark-nlp==1.7.3

In [ ]:
import pandas as pd
pd.set_option('max_colwidth', 800)

# Create a spark context that includes a 3rd party jar for NLP

In [ ]:
jarPath = "spark-nlp-assembly-1.7.3.jar"

from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .config("spark.jars.packages", "JohnSnowLabs:spark-nlp:1.8.2") \
        .getOrCreate()
spark

# Read multiple files in a dir as one Dataframe

In [ ]:
dataPath = "./data/reddit/*.json"
# read in whole json :O
df = spark.read.json(dataPath)
print(df.count())
df.printSchema()

# Deal with Struct type to query subfields 

In [ ]:
title = "data.title"
author = "data.author"

dfAuthorTitle = df.select(title, author)
dfAuthorTitle.limit(5).toPandas()

# Try to implement the equivalent of flatMap in dataframes

In [ ]:
import pyspark.sql.functions as F
# take the array and explode it, split on white spaces
dfWordCount = df.select(F.explode(F.split(title, "\\s+")).alias("word")).groupBy("word").count().orderBy(F.desc("count"))
dfWordCount.limit(10).toPandas()

# Use an NLP libary to do Part-of-Speech Tagging

In [ ]:
from com.johnsnowlabs.nlp.pretrained.pipeline.en import BasicPipeline as bp
dfAnnotated = bp.annotate(dfAuthorTitle, "title")
dfAnnotated.printSchema()

## Deal with Map type to query subfields

In [ ]:
dfPos = dfAnnotated.select("text", "pos.metadata", "pos.result")
dfPos.limit(5).toPandas()

In [ ]:
dfPos= dfAnnotated.select(F.explode("pos").alias("pos"))
dfPos.printSchema()
dfPos.toPandas()

## Keep only proper nouns NNP or NNPS

In [ ]:
nnpFilter = "pos.result = 'NNP' or pos.result = 'NNPS' "
dfNNP = dfPos.where(nnpFilter)
dfNNP.limit(10).toPandas()

## Extract columns form a map in a col

In [ ]:
dfWordTag = dfNNP.selectExpr("pos.metadata['word'] as word", "pos.result as tag")
dfWordTag.limit(10).toPandas()

In [ ]:
from pyspark.sql.functions import desc
dfWordTag.groupBy("word").count().orderBy(desc("count")).show()